In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers

In [3]:
import importlib
import audio_tools.fileutil as fileutil

In [4]:
importlib.reload(fileutil)

<module 'audio_tools.fileutil' from '../audio_tools/fileutil.py'>

In [5]:
X_train, X_test, y_train, y_test = fileutil.make_arrays(
    "../../data/ESC-50-master/audio",
    "../../data/ESC-50-master/meta/esc50.csv",
    .3,
    1
)

In [69]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(dtype=np.int32, sparse=False)
ohe.fit(y_train)

OneHotEncoder(dtype=<class 'numpy.int32'>, sparse=False)

In [73]:
y_train_enc = ohe.transform(y_train)
y_test_enc = ohe.transform(y_test)
print(f'{y_train_enc.shape} | {y_test_enc.shape}')

(280, 50) | (120, 50)


In [25]:
X_train1 = np.expand_dims(X_train, 2)
X_test1 = np.expand_dims(X_test, 2)

X_train1.shape

(280, 80000, 1)

In [87]:
X_train1[0].shape

(80000, 1)

In [95]:
input_tensor = layers.Input(shape=(80000,1))
x = layers.Conv1D(16, 32, padding='same', activation='sigmoid')(input_tensor)
f = layers.Flatten()(x)
output_tensor = layers.Dense(50, activation='softmax')(f)

model_1 = tf.keras.Model(input_tensor, output_tensor)

model_1.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.SGD(),
    metrics=['accuracy']
)

model_1.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 80000, 1)]        0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 80000, 16)         528       
_________________________________________________________________
flatten_3 (Flatten)          (None, 1280000)           0         
_________________________________________________________________
dense_20 (Dense)             (None, 50)                64000050  
Total params: 64,000,578
Trainable params: 64,000,578
Non-trainable params: 0
_________________________________________________________________


In [96]:
history = model_1.fit(
    X_train1, y_train_enc,
    validation_data=(X_test1, y_test_enc),
    epochs=100,
    verbose=1
)

Epoch 1/100
9/9 [==============================] - ETA: 0s - loss: 213.8790 - accuracy: 0.0237

ValueError: in user code:

    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py: